In [23]:
# Suzan Iloglu, Dec 11,2020
# Positive results from Pfizer, Moderna and AstraZeneca show a vaccine can work
# In this research, we wanna show the allocation of vaccine over counties given
# the number of total vaccines ordered by each state\


# Import packages
import csv
from itertools import product
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import time
import requests
import io
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.options.display.max_columns =200
from IPython.display import Image
import sodapy
from sodapy import Socrata
from collections import Counter
from scipy import stats


# MAPPING THE VACCINE ALLOCATION
The project presents multiple options for how to distribute vaccine within each county per state. It shows that what you choose to prioritize greatly impacts where vaccine would be sent. The followings are our options to choose to define vulnerability:

- Adult population

An extension of the federal government's vaccine distribution rationale from the state level to the county.

- Phase 1a

ACIP’s prioritization of healthcare personnel & long-term care facility residents

    - Phase 1a weighted by SVI (incl. race/ethnicity)
    
    Sub-allocation within Phase 1a by CDC’s Social Vulnerability Index 
    
    - Phase 1a weighted by SVI (excl. race/ethnicity)
    
	Sub-allocation within Phase 1a by CDC’s Social Vulnerability Index


### I. Importing SVI data which includes the variables for calculating county SVI for each state
The CDC uses both a USA-wide and a state by state SVI scores. For our project given that funding is likely going to be managed at a state level, using a state by state SVI scores makes the most sense and will be most sensitive to regional socioeconomic differences. Even though the CDC SVI scores are calculated using percentile rankings, the data sets include raw data estimates for each variables. The following table shows the variablaes used in the method of calculating SVI scores. 




      American Community Survey (ACS), 2015-2019 (5-year) data for the following estimates:
<img src="Input/img/SVI_comp.png" width="500">


Note: Full documentation for 2018 data is available <a href="https://svi.cdc.gov/data-and-tools-download.html">here</a> 
This part of the code shows preliminary mapping of <a href = "https://svi.cdc.gov/">the CDC's Social Vulnerability Index</a>.

Later in the notebook, we will provide the formula to create the SVI value we use in our project. First, we import the data for the US mainland and Puerto Rico.

In [24]:
## import svi data downloaded from CDC website as cited above
svi_counties =  pd.read_csv('Input/SVI_2019_State_Data.csv')
svi_counties.head(5)

,FIPS,COUNTY,STATE,E_NOHSDP,E_AGE17,E_POV,E_PCI,E_GROUPQ,E_AGE65,EP_AGE65,E_TOTPOP,EP_NOHSDP,EP_POV,E_UNINSUR,EP_UNINSUR,E_HH,E_DISABL,EP_DISABL,E_UNEMP,EP_UNEMP,E_HU,E_MOBILE,EP_MOBILE,E_NOVEH,EP_NOVEH,M_NOHSDP,M_AGE17,M_POV,M_PCI,M_GROUPQ,M_AGE65,MP_AGE65,M_TOTPOP,MP_NOHSDP,MP_POV,M_UNINSUR,MP_UNINSUR,M_HH,M_DISABL,MP_DISABL,M_UNEMP,MP_UNEMP,M_HU,M_MOBILE,MP_MOBILE,M_NOVEH,MP_NOVEH,ST,EP_PCI,MP_PCI,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_CROWD,M_CROWD,EP_AGE17,MP_AGE17,EP_SNGPNT,MP_SNGPNT,EP_MINRTY,MP_MINRTY,EP_LIMENG,MP_LIMENG,EP_MUNIT,MP_MUNIT,EP_CROWD,MP_CROWD,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,SPL_THEME3_xMin,RPL_THEME3_xMin,SPL_THEMES_xMin,RPL_THEMES_xMin,RPL_rank,RPL_xMin_rank
0,1063,Greene County,Alabama,1204,1884,3140,14884,68,1773,21.3,8324,20.8,38.1,1057,12.8,2951,2039,24.7,311,11.5,5112,1828,35.8,531,18.0,241,13.0,537,1570,32.0,NaN,NaN,NaN,4.2,6.5,260,3.1,246,318,3.8,137,5.1,29,206,4.0,177,5.8,1,14884,1570,333,107.154095,6889,NaN,4,66.452991,299,153.287312,64,51.244512,22.633349,NaN,11.284310,3.507149,82.760692,NaN,0.051640,0.857901,5.848983,2.998395,2.168756,1.727077,0.816915,NaN,0.994906,0.954955,0.972662,0.888889,3.811412,0.985673,0.751476,0.554919,0.960840,0.987902,3.255137,0.993314,0.986310,0.076421,1.062731,0.542502,0.751476,0.975148,0.584654,0.985399,0.110793,3.407469,0.920089,11.536749,0.994269,0.076421,0.076421,10.550439,0.992677,67.0,67.0
1,1005,Barbour County,Alabama,4812,5307,6875,18473,2886,4710,18.6,25361,26.8,30.7,2544,11.3,9345,4806,21.4,849,9.2,12013,3520,29.3,950,10.2,333,27.0,558,942,339.0,38.0,0.1,NaN,1.8,2.4,342,1.5,313,369,1.6,176,1.9,143,255,2.0,184,2.0,1,18473,942,444,105.801701,13743,NaN,454,170.569634,208,86.023253,360,157.175062,20.925831,NaN,4.751204,1.120935,54.189504,NaN,1.890721,0.710349,1.731458,0.715788,3.852327,1.676960,11.379677,NaN,0.976441,0.913327,0.939733,0.972939,3.802439,0.984400,0.505747,0.328558,0.882840,0.274117,1.991261,0.475326,0.896530,0.736564,1.633094,0.876472,0.335508,0.930724,0.863312,0.894377,0.947787,3.971708,0.995861,11.398502,0.992359,0.736564,0.736564,10.501972,0.991722,66.0,66.0
2,1107,Pickens County,Alabama,2680,4031,4204,23024,1829,3739,18.5,20243,18.5,22.7,1697,9.1,7637,4657,25.1,802,10.3,9588,2414,25.2,787,10.3,273,NaN,561,1440,264.0,23.0,0.1,NaN,1.9,3.0,326,1.7,279,383,2.1,204,2.5,116,254,2.7,170,2.1,1,23024,1440,535,113.877127,9383,NaN,322,132.582050,158,69.462220,100,63.126856,19.913056,NaN,7.005369,1.468997,46.351825,NaN,1.678307,0.691034,1.647893,0.724196,1.309415,0.825207,9.035222,NaN,0.885387,0.940043,0.773221,0.814072,3.412724,0.922954,0.496738,0.225406,0.966253,0.829672,2.518069,0.786692,0.857052,0.709848,1.566900,0.846864,0.320596,0.886300,0.277726,0.896552,0.918179,3.299353,0.886979,10.797046,0.974530,0.709848,0.709848,9.939994,0.973575,64.0,65.0
3,1131,Wilcox County,Alabama,1675,2577,3111,16841,251,2026,19.0,10681,23.5,30.1,1134,10.9,3854,2001,19.2,541,15.3,5777,2333,40.4,600,15.6,204,NaN,417,1299,108.0,NaN,NaN,NaN,2.9,4.1,221,2.1,213,272,2.6,152,3.8,44,196,3.4,131,3.1,1,16841,1299,218,75.802375,7814,NaN,65,115.585466,8,23.021729,54,55.713553,24.126954,NaN,5.656461,1.941846,73.157944,NaN,0.649351,NaN,0.138480,0.398505,1.401142,1.443528,2.349967,NaN,0.972620,0.979807,0.958993,0.939191,3.850612,0.990767,0.548618,0.758994,0.779370,0.553327,2.640308,0.841452,0.972620,0.453557,1.426177,0.767590,0.036347,0.990991,0.316869,0.974837,0.579752,2.898795,0.701687,10.815893,0.975486,0.453557,0.453557,9.843272,0.967845,65.0,64.0
4,1105,Perry County,Alabama,1010,2007,2612,15055,810,1772,19.1,9293,17.2,30.8,846,9.3,3070,2016,22.1,290,10.1,4736,1153,24.3,399,13.0,277,NaN,669,2347,200.0,NaN,NaN,NaN,4.7,7.8,283,3.1,314,337,3.7,184,6.2,75,251,5.3,180,5.6,1,15055,2347,225,113.225439,6566,

In [25]:
## Create the list for State
S = svi_counties.STATE.unique().tolist()
if "0" in S:
    S.remove(0)
State = [str(s).strip() for s in S]

In [26]:
## Replacing -999 values with 0 for calculations

svi_county = svi_counties.fillna(0)
svi_county  = svi_county.replace(-999, 0)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)


In [27]:
# Create a dictionary for the states of the given the county FIPS
county_of_states = dict(zip(svi_county.FIPS, svi_county.STATE))

# Create a dictionary for the name of the given the county FIPS
county_name = dict(zip(svi_county.FIPS, svi_county.COUNTY))

# Create the list for county FIPS, we consider counties as analogy to the center for community health workers
location = svi_county.FIPS.tolist() #[k for k in SVI_county] #[9001, 9003, 9005, 9007, 9009, 9011, 9013, 9015]#[k for k in SVI_county]


In [28]:
cartesian_pro_county_state = [(i,county_of_states[i]) for i in location ]


In [29]:
# Create a seperate dictionary for the variables to calculate SVI

# Persons below poverty estimate, 2014-2018 ACS
E_POV = dict(zip(svi_county.FIPS, svi_county.EP_POV))

# Civilian (age 16+) unemployed estimate, 2014-2018 ACS
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.EP_UNEMP))

# Per capita income estimate, 2014-2018 ACS
E_PCI = dict(zip(svi_county.FIPS, svi_county.EP_PCI/1000))

# Persons (age 25+) with no high school diploma estimate, 2014-2018 ACS
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.EP_NOHSDP))

# Persons aged 65 and older estimate
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.EP_AGE65))

# Persons aged 17 and younger estimate
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.EP_AGE17))

# Population with a disability estimate
E_DISABL = dict(zip(svi_county.FIPS, svi_county.EP_DISABL))

# Single parent households with children under 18 estimate
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.EP_SNGPNT))

# Minority (all persons except white, nonHispanic) estimate, 2014-2018 ACS
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.EP_MINRTY))

# Persons (age 5+) who speak English "less than well" estimate, 2014-2018 ACS
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.EP_LIMENG))

# Housing in structures with 10 or more units estimate, 2014-2018 ACS
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.EP_MUNIT))

# Mobile homes estimate MOE, 2014-2018 ACS
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.EP_MOBILE))

# At household level (occupied housing units), more people than rooms estimate, 2014-2018 ACS
E_CROWD = dict(zip(svi_county.FIPS, svi_county.EP_CROWD))

# Households with no vehicle available estimate, 2014-2018 ACS
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.EP_NOVEH))

# Persons in institutionalized group quarters estimate, 2014-2018 ACS
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.EP_GROUPQ))

# Percentage of persons below poverty estimate
E_POV = dict(zip(svi_county.FIPS, svi_county.EP_POV))

In [30]:

df_a = pd.read_csv("Input/ACSST5Y2019.S0101_data_with_overlays_2020-12-15T094007.csv", header=[1])
df_a.head(1)
df_a = df_a.rename(columns = {"Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!18 years and over":'Adult_pop'})
#Adult population
Adult_pop_county = dict(zip(df_a['FIPS'], df_a["Adult_pop"]))


In [31]:
Adult_pop_county

{1001: 42175,
 1003: 166595,
 1005: 20054,
 1007: 17862,
 1009: 44292,
 1011: 8120,
 1013: 15373,
 1015: 89666,
 1017: 26595,
 1019: 20658,
 1021: 33420,
 1023: 10322,
 1025: 18841,
 1027: 10559,
 1029: 11533,
 1031: 39403,
 1033: 43218,
 1035: 9790,
 1037: 8956,
 1039: 29037,
 1041: 10738,
 1043: 64148,
 1045: 37892,
 1047: 29720,
 1049: 53800,
 1051: 62915,
 1053: 28769,
 1055: 80579,
 1057: 13014,
 1059: 23582,
 1061: 20599,
 1063: 6440,
 1065: 11355,
 1067: 13537,
 1069: 80349,
 1071: 40870,
 1073: 508503,
 1075: 10835,
 1077: 74247,
 1079: 25875,
 1081: 126764,
 1083: 73144,
 1085: 7808,
 1087: 15460,
 1089: 282589,
 1091: 14918,
 1093: 23592,
 1095: 71756,
 1097: 316303,
 1099: 16522,
 1101: 173501,
 1103: 91846,
 1105: 7286,
 1107: 16212,
 1109: 26863,
 1111: 17751,
 1113: 43783,
 1115: 67948,
 1117: 162807,
 1119: 10327,
 1121: 63130,
 1123: 32072,
 1125: 163687,
 1127: 49854,
 1129: 12753,
 1131: 8104,
 1133: 18858,
 2013: 2901,
 2016: 4716,
 2020: 221655,
 2050: 11740,
 2060:

# Phase 1a population
ACIP’s prioritization of healthcare personnel & long-term care facility residents

In [32]:
#Phase 1a population includes the number of health care workers and long-term care facility residents
first_phase = pd.read_csv("Input/Phase_1a_pop.csv")
first_phase = first_phase.fillna(0)
#Set FIPS type as int
first_phase['FIPS'] = first_phase['FIPS'].astype(int)


In [33]:
first_phase.head(80)

,FIPS,NAME,hp_emp,ltcf_res,phase_1a
0,1001,"Autauga County, Alabama",2288,120.0,2408.0
1,1003,"Baldwin County, Alabama",9922,536.0,10458.0
2,1005,"Barbour County, Alabama",658,143.0,801.0
3,1007,"Bibb County, Alabama",951,119.0,1070.0
4,1009,"Blount County, Alabama",2613,202.0,2815.0
5,1011,"Bullock County, Alabama",188,95.0,283.0
6,1013,"Butler County, Alabama",821,147.0,968.0
7,1015,"Calhoun County, Alabama",4974,544.0,5518.0
8,1017,"Chambers County, Alabama",1414,266.0,1680.0
9,1019,"Cherokee County, Alabama",1343,160.0,1503.0


In [34]:
#Create a dictionary for the Phase 1a population
Firstphase_county = dict(zip(first_phase.FIPS, first_phase.phase_1a))

In [35]:
###############################################################################################
######################## END calculating different types of vulnerabilities ###################

In [36]:
Firstphase_county

{1001: 2408.0,
 1003: 10458.0,
 1005: 801.0,
 1007: 1070.0,
 1009: 2815.0,
 1011: 283.0,
 1013: 968.0,
 1015: 5518.0,
 1017: 1680.0,
 1019: 1503.0,
 1021: 1601.0,
 1023: 502.0,
 1025: 990.0,
 1027: 684.0,
 1029: 593.0,
 1031: 2955.0,
 1033: 3031.0,
 1035: 380.0,
 1037: 626.0,
 1039: 1843.0,
 1041: 621.0,
 1043: 4770.0,
 1045: 1915.0,
 1047: 1731.0,
 1049: 3370.0,
 1051: 4081.0,
 1053: 1628.0,
 1055: 6020.0,
 1057: 966.0,
 1059: 1048.0,
 1061: 1487.0,
 1063: 271.0,
 1065: 727.0,
 1067: 930.0,
 1069: 6868.0,
 1071: 2125.0,
 1073: 45387.0,
 1075: 656.0,
 1077: 4930.0,
 1079: 1267.0,
 1081: 7229.0,
 1083: 3694.0,
 1085: 393.0,
 1087: 800.0,
 1089: 18989.0,
 1091: 1057.0,
 1093: 1646.0,
 1095: 3976.0,
 1097: 23079.0,
 1099: 977.0,
 1101: 11290.0,
 1103: 6201.0,
 1105: 436.0,
 1107: 969.0,
 1109: 1228.0,
 1111: 998.0,
 1113: 2889.0,
 1115: 4427.0,
 1117: 10968.0,
 1119: 561.0,
 1121: 3687.0,
 1123: 2396.0,
 1125: 12080.0,
 1127: 3305.0,
 1129: 713.0,
 1131: 423.0,
 1133: 1127.0,
 2013: 0.0,


Since we allocate tha vaccine proportional to the county values of certain vulnaribilites within state, we need a few function to help us with the calculations. 



In [37]:
# This function return the value for the state for the given dictionary

# More specifically sum up the values for the counties of each state

def total_state(dict_1):
    state_dict = {}

    for s in State:
        state_dict [s] = sum(float(dict_1[j]) for j in dict_1 if (j,s) in cartesian_pro_county_state)  
    return state_dict



In [38]:
Adult_pop_state = total_state(Adult_pop_county)


In [39]:
Adult_pop_state

{'Alabama': 3779874.0,
 'Alaska': 552674.0,
 'Arizona': 5414955.0,
 'Arkansas': 2295102.0,
 'California': 30261351.0,
 'Colorado': 4349344.0,
 'Connecticut': 2831241.0,
 'Delaware': 753564.0,
 'District of Columbia': 568753.0,
 'Florida': 16719174.0,
 'Georgia': 7898607.0,
 'Hawaii': 1117456.0,
 'Idaho': 1276603.0,
 'Illinois': 9879105.0,
 'Indiana': 5093212.0,
 'Iowa': 2410771.0,
 'Kansas': 2199582.0,
 'Kentucky': 3439746.0,
 'Louisiana': 3561012.0,
 'Maine': 1082994.0,
 'Maryland': 4677166.0,
 'Massachusetts': 5479293.0,
 'Michigan': 7787387.0,
 'Minnesota': 4267530.0,
 'Mississippi': 2270925.0,
 'Missouri': 4723298.0,
 'Montana': 822263.0,
 'Nebraska': 1440464.0,
 'Nevada': 2291340.0,
 'New Hampshire': 1088160.0,
 'New Jersey': 6916982.0,
 'New Mexico': 1603978.0,
 'New York': 15463820.0,
 'North Carolina': 7968262.0,
 'North Dakota': 580973.0,
 'Ohio': 9050387.0,
 'Oklahoma': 2975906.0,
 'Oregon': 3261860.0,
 'Pennsylvania': 10129139.0,
 'Rhode Island': 849887.0,
 'South Carolina':

In [40]:
Firstphase_State = total_state(Firstphase_county)


In [41]:
vac = pd.read_csv('https://data.cdc.gov/resource/saz5-9hgg.csv')
vac['jurisdiction'] = vac['jurisdiction'].str.replace("*", "")
vac.dropna(subset = ['total_pfizer_allocation_first_dose_shipments'], inplace=True)
vac['first_doses'] = vac['total_pfizer_allocation_first_dose_shipments'].str.replace(",", "")
vac['first_doses'] = vac['first_doses'].astype(int)
vac.head(5)



,jurisdiction,hhs_region,first_doses_12_14,second_doses_shipment_12_14,doses_allocated_week_of_12_21,second_dose_shipment_12_17,doses_allocated_week_of_12_28,second_dose_shipment_12_28,doses_allocated_week_of_01_04,second_dose_shipment_01_04,doses_allocated_week_of_01_10,second_dose_shipment_week_of_01_10,doses_distribution_week_of_01_18,second_dose_shipment_week_of_01_18,doses_allocated_week_of_01_25,second_dose_shipment_week_of_01_25,total_pfizer_allocation_first_dose_shipments,total_allocation_pfizer_second_dose_shipments,first_doses
0,Connecticut,Region 1,"31,200","31,200","22,425","22,425","28,275","28,275","22,425","22,425","22,425","22,425","23,400","23,400","23,400","23,400","173,550","173,550",173550
1,Maine,Region 1,"12,675","12,675","8,775","8,775","10,725","10,725","8,775","8,775","8,775","8,775","8,775","8,775","8,775","8,775","67,275","67,275",67275
2,Massachusetts,Region 1,"59,475","59,475","42,900","42,900","54,600","54,600","42,900","42,900","42,900","42,900","43,875","43,875","43,875","43,875","330,525","330,525",330525
3,New Hampshire,Region 1,"12,675","12,675","8,775","8,775","10,725","10,725","8,775","8,775","8,775","8,775","8,775","8,775","8,775","8,775","67,275","67,275",67275
4,Rhode Island,Region 1,"9,750","9,750","6,825","6,825","8,775","8,775","6,825","6,825","6,825","6,825","6,825","6,825","6,825","6,825","52,650","52,650",52650


In [42]:

Jur_state = {'New York City': 'New York', 'Philadelphia': 'Pennsylvania', 'Chicago':'Illinois'}

In [43]:

Vaccine_budget_state = {s:0 for s in State}

Vaccine_budget_st =  dict(zip(vac.jurisdiction, vac.first_doses) )

sum_v = 0
for s in Vaccine_budget_st:
    
    if s in State:
        Vaccine_budget_state[s] = Vaccine_budget_st[s]
        sum_v += Vaccine_budget_st[s]
    
    if s in Jur_state:    
        Vaccine_budget_state[Jur_state[s]] += Vaccine_budget_st[s]
        sum_v += Vaccine_budget_st[s]
        
print (sum_v)

15340650


# SVI calculation 

We calculate the ratio of county value to state value by population for each SVI variables (we use EP-estimate percentage- values in the CDC data set), then we take the average of all 15 SVI variables. 

Let SVI variable set be K, where  

K = { Below Poverty, Unemployed, Income, No High School Diploma, Aged 65 or Older, Aged 17 or Younger, Civilian with a Disability, Single-Parent Households, Minority, Speaks English “Less than Well”, Multi-Unit Structures, Mobile Homes, Crowding, No Vehicle, Group Quarters }

We use the estimate percentage of these variables in a county base. To calculate the SVI value for each county, we take the average of the estimate percentage of these 15 variables.

Let $S$ is the set of states and $j$ is a county in the state $s$, where $s \in S$, $c^k_j$ SVI variable $k \in K$ value for county j, and $c_s$ SVI variable value for state s.

$SVI_j = \frac{1}{15}\sum_{k \in K} c^k_j$


In [44]:
# Sum all SVI variable estimated percentage values for each county

SVI_county_sum = dict(Counter(E_POV) + Counter(E_UNEMP) + Counter(E_PCI) + Counter(E_NOHSDP) + Counter(E_AGE65) + Counter(E_AGE17) + Counter(E_DISABL) + Counter(E_SNGPNT) + Counter(E_MINRTY) + Counter(E_LIMENG) + Counter(E_MUNIT) + Counter(E_MOBILE) + Counter(E_CROWD) + Counter(E_NOVEH) + Counter(E_GROUPQ))

# Divide the sum of all SVI variable values
SVI_county = {j: (SVI_county_sum[j]/(15*100)) for j in SVI_county_sum }


# SVI calculation (excl. race/ethnicity)

We calculate the ratio of county value to state value by population for each SVI variables (we use EP-estimate percentage- values in the CDC data set), then we take the average of all 15 SVI variables. 

Let SVI variable set be K, where  

K = { Below Poverty, Unemployed, Income, No High School Diploma, Aged 65 or Older, Aged 17 or Younger, Civilian with a Disability, Single-Parent Households, Speaks English “Less than Well”, Multi-Unit Structures, Mobile Homes, Crowding, No Vehicle, Group Quarters }

We use the estimate percentage of these variables in a county base. To calculate the SVI value for each county, we take the average of the estimate percentage of these 14 (Minority excluded) variables.

Let $S$ is the set of states and $j$ is a county in the state $s$, where $s \in S$, $c^k_j$ SVI variable $k \in K$ value for county j, and $c_s$ SVI variable value for state s.

$SVI_j = \frac{1}{14}\sum_{k \in K} c^k_j$

In [45]:
# Sum all SVI variable estimate percentage values (except Minortity) for each county

SVI_county_sum_no_race = dict(Counter(E_POV) + Counter(E_UNEMP) + Counter(E_PCI) + Counter(E_NOHSDP) + Counter(E_AGE65) + Counter(E_AGE17) + Counter(E_DISABL) + Counter(E_SNGPNT) + Counter(E_LIMENG) + Counter(E_MUNIT) + Counter(E_MOBILE) + Counter(E_CROWD) + Counter(E_NOVEH) + Counter(E_GROUPQ))


# Divide the sum of all SVI variable values
SVI_county_no_race = {j: (SVI_county_sum_no_race[j]/(14*100)) for j in SVI_county_sum }


# Proportional Vaccine Allocation

We consider allocating vaccines ordered by each state to its counties proportional to multiple variables. 

- Adult population

An extension of the federal government's vaccine distribution rationale from the state level to the county.

- Phase 1a

ACIP’s prioritization of healthcare personnel & long-term care facility residents

    - Phase 1a weighted by SVI (incl. race/ethnicity)
    
    Sub-allocation within Phase 1a by CDC’s Social Vulnerability Index 
    
    - Phase 1a weighted by SVI (excl. race/ethnicity)
    
	Sub-allocation within Phase 1a by CDC’s Social Vulnerability Index


To calculate the total number of allocated vaccines to per county according to these vulnerability criterias, we define the following function called "Proportional_allocation", in which we multiply the total amount of vaccine for each state with the ratio of the chosen vulnerability criteria of the county to the chosen vulnerability criteria of the state, the function return a dictionary with the counties as keys and the number of vaccines allocated to each county for the chosen vulnerability criteria as values. 

<a href="https://covid.cdc.gov/covid-data-tracker/#vaccinations">CDC website</a>  provides the total number of distributed vaccine in each state. 


In [46]:
def Proportional_allocation(county_level, state_level, state_budget):
    prop_allocate = {}
    

    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            
            prop_allocate[j,s] = (float(county_level[j])/float(state_level[s]))*float(state_budget[s])
        
        else:
            prop_allocate[j,s] = 0
            
    
    return prop_allocate

In [47]:
def total_state_pop(dict_1):
    state_dict = {}

    for s in State:
        state_dict [s] = sum(float(dict_1[j]*Firstphase_county[j]) for j in dict_1 if (j,s) in cartesian_pro_county_state)  
    return state_dict



In [48]:
def Proportional_allocation_pop(county_level, state_budget):
    prop_allocate = {}
    state_level = total_state_pop(county_level)
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            
            prop_allocate[j,s] = ((float(county_level[j])*Firstphase_county[j])/(float(state_level[s])))*float(state_budget[s])
        
        else:
            prop_allocate[j,s] = 0
            
    
    return prop_allocate

# Proportional allocation for different vulnerability values

Let V = {Adult population, Phase 1a population, SVI weighted Phase 1a population, SVI (excl. race/ethnicity) weighted Phase 1a population}. We assume $v_j$ represent the vulnerability value for county $j \in J$, while $v_s$ represent the sum of the vulnerability values for each county in the state of county j. $Vac_s$ represents the total number of vaccine state s ordered. 

$Prop_{v_j} = \frac{v_j}{v_s}*Vac_s$



In [49]:
# Calling proportional allocation function for different vulnerability criterias

# Proportional allocation according to SVI score in each county
Proportional_to_Adult_pop = Proportional_allocation(Adult_pop_county, Adult_pop_state, Vaccine_budget_state)

# Proportional allocation according to SVI score in each county
Proportional_to_Firstphase = Proportional_allocation(Firstphase_county, Firstphase_State, Vaccine_budget_state)

# Proportional allocation according to SVI score in each county
Proportional_to_SVI = Proportional_allocation_pop(SVI_county, Vaccine_budget_state)

# Proportional allocation according to YPLL in each county
Proportional_to_SVI_no_race = Proportional_allocation_pop(SVI_county_no_race, Vaccine_budget_state)


In [50]:
for (j,s) in cartesian_pro_county_state:
    print (j,Firstphase_county[j], Firstphase_State[s], Vaccine_budget_state[s])

1063 271.0 257045.0 227175
1005 801.0 257045.0 227175
1107 969.0 257045.0 227175
1131 423.0 257045.0 227175
1105 436.0 257045.0 227175
1065 727.0 257045.0 227175
1119 561.0 257045.0 227175
1123 2396.0 257045.0 227175
1085 393.0 257045.0 227175
1047 1731.0 257045.0 227175
1121 3687.0 257045.0 227175
1087 800.0 257045.0 227175
1059 1048.0 257045.0 227175
1023 502.0 257045.0 227175
1093 1646.0 257045.0 227175
1025 990.0 257045.0 227175
1133 1127.0 257045.0 227175
1113 2889.0 257045.0 227175
1021 1601.0 257045.0 227175
1053 1628.0 257045.0 227175
1127 3305.0 257045.0 227175
1075 656.0 257045.0 227175
1039 1843.0 257045.0 227175
1017 1680.0 257045.0 227175
1061 1487.0 257045.0 227175
1057 966.0 257045.0 227175
1109 1228.0 257045.0 227175
1091 1057.0 257045.0 227175
1007 1070.0 257045.0 227175
1049 3370.0 257045.0 227175
1011 283.0 257045.0 227175
1015 5518.0 257045.0 227175
1111 998.0 257045.0 227175
1045 1915.0 257045.0 227175
1101 11290.0 257045.0 227175
1035 380.0 257045.0 227175
1129 71

18171 466.0 403389.0 307125
18127 10558.0 403389.0 307125
18081 10546.0 403389.0 307125
18183 2113.0 403389.0 307125
18173 4847.0 403389.0 307125
18063 11329.0 403389.0 307125
18129 1442.0 403389.0 307125
18059 4824.0 403389.0 307125
18011 4365.0 403389.0 307125
18057 21117.0 403389.0 307125
19047 993.0 214471.0 147225
19179 2354.0 214471.0 147225
19053 387.0 214471.0 147225
19007 644.0 214471.0 147225
19021 1101.0 214471.0 147225
19127 2860.0 214471.0 147225
19185 406.0 214471.0 147225
19039 568.0 214471.0 147225
19175 844.0 214471.0 147225
19193 6235.0 214471.0 147225
19057 2300.0 214471.0 147225
19177 378.0 214471.0 147225
19111 2280.0 214471.0 147225
19101 1205.0 214471.0 147225
19187 2469.0 214471.0 147225
19115 675.0 214471.0 147225
19067 1042.0 214471.0 147225
19005 1145.0 214471.0 147225
19139 2334.0 214471.0 147225
19043 1245.0 214471.0 147225
19145 1074.0 214471.0 147225
19051 496.0 214471.0 147225
19013 9196.0 214471.0 147225
19155 6097.0 214471.0 147225
19029 1000.0 214471.

28029 1500.0 169696.0 137475
28083 1454.0 169696.0 137475
28159 934.0 169696.0 137475
28101 1182.0 169696.0 137475
28005 722.0 169696.0 137475
28115 1512.0 169696.0 137475
28153 737.0 169696.0 137475
28035 4799.0 169696.0 137475
28113 1984.0 169696.0 137475
28013 712.0 169696.0 137475
28119 272.0 169696.0 137475
28147 558.0 169696.0 137475
28097 522.0 169696.0 137475
28041 485.0 169696.0 137475
28043 1413.0 169696.0 137475
28047 10178.0 169696.0 137475
28027 1405.0 169696.0 137475
28017 764.0 169696.0 137475
28067 3835.0 169696.0 137475
28039 1098.0 169696.0 137475
28107 1853.0 169696.0 137475
28127 1648.0 169696.0 137475
28137 1507.0 169696.0 137475
28141 921.0 169696.0 137475
28075 5916.0 169696.0 137475
28161 585.0 169696.0 137475
28149 2334.0 169696.0 137475
28145 1310.0 169696.0 137475
28065 720.0 169696.0 137475
28049 16855.0 169696.0 137475
28139 1256.0 169696.0 137475
28045 1958.0 169696.0 137475
28109 2685.0 169696.0 137475
28009 312.0 169696.0 137475
28111 515.0 169696.0 1374

42101 124545.0 917696.0 613275
42033 5794.0 917696.0 613275
42051 8953.0 917696.0 613275
42105 982.0 917696.0 613275
42085 8820.0 917696.0 613275
42063 5354.0 917696.0 613275
42039 6001.0 917696.0 613275
42059 2387.0 917696.0 613275
42049 21247.0 917696.0 613275
42031 2915.0 917696.0 613275
42035 2311.0 917696.0 613275
42111 4701.0 917696.0 613275
42097 8500.0 917696.0 613275
42079 22253.0 917696.0 613275
42023 226.0 917696.0 613275
42061 2645.0 917696.0 613275
42067 1247.0 917696.0 613275
42083 3485.0 917696.0 613275
42073 6362.0 917696.0 613275
42121 3582.0 917696.0 613275
42021 10431.0 917696.0 613275
42113 520.0 917696.0 613275
42087 3464.0 917696.0 613275
42015 4481.0 917696.0 613275
42075 10490.0 917696.0 613275
42107 10162.0 917696.0 613275
42011 26653.0 917696.0 613275
42069 15612.0 917696.0 613275
42077 25701.0 917696.0 613275
42065 3176.0 917696.0 613275
42119 2849.0 917696.0 613275
42117 2504.0 917696.0 613275
42013 9840.0 917696.0 613275
42127 2862.0 917696.0 613275
42071 3

54031 762.0 118557.0 89700
54003 6122.0 118557.0 89700
54051 2202.0 118557.0 89700
54065 1163.0 118557.0 89700
54037 2744.0 118557.0 89700
54079 3941.0 118557.0 89700
55001 731.0 383462.0 273975
55078 0.0 383462.0 273975
55003 1012.0 383462.0 273975
55079 67480.0 383462.0 273975
55019 2161.0 383462.0 273975
55053 1134.0 383462.0 273975
55041 419.0 383462.0 273975
55123 2173.0 383462.0 273975
55137 0.0 383462.0 273975
55057 1705.0 383462.0 273975
55023 869.0 383462.0 273975
55103 1158.0 383462.0 273975
55067 967.0 383462.0 273975
55075 2705.0 383462.0 273975
55059 10180.0 383462.0 273975
55047 882.0 383462.0 273975
55051 437.0 383462.0 273975
55081 3023.0 383462.0 273975
55113 828.0 383462.0 273975
55101 11915.0 383462.0 273975
55077 725.0 383462.0 273975
55107 793.0 383462.0 273975
55005 3019.0 383462.0 273975
55105 9993.0 383462.0 273975
55119 1078.0 383462.0 273975
55099 669.0 383462.0 273975
55031 3835.0 383462.0 273975
55121 1874.0 383462.0 273975
55043 2851.0 383462.0 273975
55037

# Percentile Rank

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.percentileofscore.html"> The function scipy.stats.percentileofscore (a, score, kind='rank')   </a>
computes the percentile rank of a score relative to a list of scores. 
"rank": Average percentage ranking of score. In case of multiple matches, average the percentage rankings of all matching scores.

In [51]:
# Calculate percentile ranks

def percentile_ranks(data):
    x = {s: [] for s in State}

    for (j,s) in cartesian_pro_county_state:
         
        x[s].append(data[j])  
    
    percentile_ranks = {i: stats.percentileofscore(x[s], data[i], 'rank') for (i,s) in cartesian_pro_county_state}

    return percentile_ranks

In [52]:
# Write timestamp 

time_stamp = time.strftime('%m-%d-%Y %H:%M:%S')
with open('Output/time_stamp.csv','w') as f:
    w = csv.writer(f)
    now = time.strftime('%m/%d/%Y %H:%M:%S')
    w.writerow(['time',now])
    

In [53]:
#Write a function to order the dicts
def order_k(dict_1):
    dict_2 = {}
    for m in location:
        if m in dict_1.keys():
            dict_2[m] = dict_1[m]
        else:
            dict_2[m] = 0
    
    return dict_2
            

In [54]:
Adult_pop_county = order_k(Adult_pop_county)
Firstphase_county = order_k(Firstphase_county)
SVI_county = order_k(SVI_county)
SVI_county_no_race = order_k(SVI_county_no_race)

In [55]:
# Write file allocation with each strategies for each county 

Strategies = ["Adult_pop", "Firstphase", "SVI", "SVI_no_race"]

fieldnames = []  
fieldnames.append('County_FIPS')


SVI_values = {i:SVI_county[i] for i in location}
s_count = 1
for s in Strategies:   
    fieldnames.append('Proportional_allocation_to_' + s)
    fieldnames.append(s)
    fieldnames.append('Percentile_ranks_' + s)

    
        

writefile = 'Output/County_level_proportional_vaccine_allocation.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(fieldnames)
    for row in zip(location
                   , Proportional_to_Adult_pop.values(),        Adult_pop_county.values(),     percentile_ranks(Adult_pop_county).values()
                   , Proportional_to_Firstphase.values(),       Firstphase_county.values(),    percentile_ranks(Firstphase_county).values()
                   , Proportional_to_SVI.values(),              SVI_county.values(),           percentile_ranks(SVI_county).values()
                   , Proportional_to_SVI_no_race.values(),      SVI_county_no_race.values(),   percentile_ranks(SVI_county_no_race).values()
                    ):                    
       
        writer.writerow(row)

In [56]:
Vaccine_state = {s: Vaccine_budget_state[s] for s in State}

In [58]:
Vaccine_state

{'Alabama': 227175,
 'Alaska': 62400,
 'Arizona': 323700,
 'Arkansas': 141375,
 'California': 1806675,
 'Colorado': 259350,
 'Connecticut': 173550,
 'Delaware': 47775,
 'District of Columbia': 37050,
 'Florida': 989625,
 'Georgia': 469950,
 'Hawaii': 70200,
 'Idaho': 77025,
 'Illinois': 471900,
 'Indiana': 307125,
 'Iowa': 147225,
 'Kansas': 136500,
 'Kentucky': 210600,
 'Louisiana': 218400,
 'Maine': 67275,
 'Maryland': 281775,
 'Massachusetts': 330525,
 'Michigan': 468000,
 'Minnesota': 257400,
 'Mississippi': 137475,
 'Missouri': 285675,
 'Montana': 52650,
 'Nebraska': 89700,
 'Nevada': 137475,
 'New Hampshire': 67275,
 'New Jersey': 416325,
 'New Mexico': 99450,
 'New York': 933075,
 'North Carolina': 474825,
 'North Dakota': 37050,
 'Ohio': 545025,
 'Oklahoma': 181350,
 'Oregon': 195975,
 'Pennsylvania': 613275,
 'Rhode Island': 52650,
 'South Carolina': 234975,
 'South Dakota': 43875,
 'Tennessee': 312000,
 'Texas': 1238250,
 'Utah': 129675,
 'Vermont': 35100,
 'Virginia': 394875

In [57]:
writefile = 'Output/State_level_vaccine_allocation.csv'

cl = ['State', 'Vaccine_allocation']
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(cl)
    for row in zip( State, Vaccine_state.values()):
        writer.writerow(row)